In [ ]:
39.334144
29        70.122162
59       100.716466
130       169.44795
149      188.279637
157      198.231465
249      287.153206
260      296.825728
301      337.139201
331      365.217019
347       384.87475
414       453.38231
437      473.932208
637      678.914027
771      826.961167
826      876.776673
834      886.634436

In [ ]:
a = [39.33414398157326, 70.12216157854493, 100.7164657981932, 169.44794962476283, 188.27963668828414, 198.2314651355102, 287.1532060019069, 296.82572790443015, 337.139200933609, 365.2170186910594, 384.8747498890809, 453.38230957020005, 473.93220815043475, 678.9140270340689, 826.9611673151898, 876.7766734624718, 886.6344355743724, 906.4780109994053, 935.9682906695742, 966.2939706588127, 986.1395888948678, 1006.3566874609425, 1046.0601474007156, 1085.4482997913778, 1095.0181543645515, 1172.4844537018682, 1191.9019739646756, 1319.4617965459297, 1339.7978852378394, 1388.6349786685926, 1407.5910164471884, 1417.8204270133008, 1437.8908334551081, 1477.086425817262, 1635.6439625772896, 1645.3586005295801, 1774.6980429431815, 1833.608480366836, 1872.3881856484759, 1892.344400133103, 1931.428408694175, 2020.7695513635977]
a = a[0:10]
print(a)

In [ ]:
%matplotlib widget
import spikeinterface.full as si
import ipywidgets as widgets
import numpy as np

# --- Load recording ---
recording =  si.read_spikeglx("E:/Florian_paper/Florian/Data/Opto_2/O5_1",stream_id='imec0.ap', load_sync_channel=False)
rec1 = si.highpass_filter(recording, freq_min=400.)
rec1 = si.phase_shift(rec1)

rec1= si.common_reference(rec1, operator="median", reference="global")
fs = recording.get_sampling_frequency()

timepoints = a
window_start_ms = -1# e.g., -10 ms before event
window_end_ms = 8 # e.g., +20 ms after event
window_start_s = window_start_ms / 1000
window_end_s = window_end_ms / 1000

# --- Build dict of sliced recordings ---
recording_slices = {}
for i, t in enumerate(timepoints):
    start_sample = max(int((t + window_start_s) * fs), 0)
    end_sample = int((t + window_end_s) * fs)
    sliced = recording.frame_slice(start_frame=start_sample, end_frame=end_sample)
    label = f"timepoint_{i+1} ({t:.2f}s)"
    recording_slices[label] = sliced


# --- UI elements ---
dropdown = widgets.Dropdown(
    options=recording_slices.keys(),
    value=list(recording_slices.keys())[0],
    description='Timepoint:'
)

output = widgets.Output()

# --- Plot update function ---
def update_plot(change):
    output.clear_output(wait=True)
    with output:
        si.plot_traces(
            recording_slices[change['new']],
            backend='ipywidgets',
            time_range=(window_start_s, window_end_s)
        )

dropdown.observe(update_plot, names='value')

# --- Show everything ---
display(dropdown, output)
update_plot({'new': dropdown.value})


In [ ]:
!phy template-gui E:\Florian_paper\Florian\Data\Opto_2\O5_1\sorted\phy\params.py